# Executing Quantum Gradients

To do *Quantum Machine Learning* (QML), we need a gradient descent strategy. For this, we define a final objective (or cost) function $C$ which is to be optimized with respect to some free parameters. As explained in the [Key concepts](https://pennylane.readthedocs.io/en/latest/concepts/autograd_quantum.html#), PennyLane incorporates both analytic differentiation, as well as numerical methods (such as the method of finite differences). Both of these are done automatically.

PennyLane is the first software library to support **automatic differentiation of quantum circuits**. Internally, it uses *classical linear combination of unitaries* or "parameter shift" trick to compute derivatives, i.e. it evaluates derivatives as the difference of two circuit evaluations with shifted parameters.

$$C(\theta_1, \theta_2) = \texttt{circuit}(\theta_1, \theta_2)$$

$$\partial_{\theta_1} C = a\big[ \texttt{circuit}(\theta_1+s, \theta_2) - \texttt{circuit}(\theta_1 - s, \theta_2) \big]$$

**NOTE:** The values of the shift and scale parameters $s$ and $a$ typically depend **only** on the *type* of gate, and not its location.

By using this method, PennyLane provides a hardware-scalable way to compute gradients and to optimize quantum circuits for QML.

### Calculating quantum gradients

Let's initialize a `device` and define a Quantum function in a `QNode`

In [1]:
# first we import the essentials
import pennylane as qml
from pennylane import numpy as np

In [2]:
dev1 = qml.device('default.qubit', wires = 1)

@qml.qnode(dev1)
def circuit(params):
    qml.RX(params[0], wires = 0)
    qml.RY(params[1], wires = 0)
    return qml.expval.PauliZ(0)

The gradient of the function `circuit` encapsulated within the `QNode` can be evaluated by utilizing the same quantum device (`dev1`) that we used to evaluate the function itself.

We can differentiate by using the built-in [grad( )](https://pennylane.readthedocs.io/en/latest/code/init.html#pennylane.grad) function.

In [3]:
dcircuit = qml.grad(circuit, argnum = 0)

The function `grad()` itself **returns a function** representing the derivative of the `QNode` with respect to the argument specified in `argnum`. 

In the example above, the function `circuit` takes one argument `params`. Hence, we specify `argnum=0`. As the argument has two elements, the returned gradient is two-dimensional. We can then evaluate this gradient function at any point in the parameter space.

In [4]:
dcircuit([0.54, 0.12])

[-0.5104386525165021, -0.10267819945693196]

**NOTE:** Quantum circuit functions, being a restricted subset of Python functions, can also make use of multiple positional arguments and keyword arguments. 

For example, we could have defined the above quantum circuit function using two positional arguments, instead of an array of arguments:

In [5]:
@qml.qnode(dev1)
def circuit2(phi1, phi2):
    qml.RX(phi1, wires = 0)
    qml.RY(phi2, wires = 0)
    return qml.expval.PauliZ(0)

In this case, `argnum=0` will return the gradient with respect to only the first parameter `phi1` and `argnum=1` will give the gradient with respect to `phi2`. To get the gradient with respect to both parameters, we can use `argnum=[0,1]`:

In [6]:
dcircuit2 = qml.grad(circuit2, argnum = [0, 1])

In [7]:
dcircuit2(0.54, 0.12)

(array(-0.51043865), array(-0.1026782))

<div class="admonition note">

<div class="admonition-title">NOTE</div>

PennyLane does **not** differentiate QNodes with respect to keyword arguments. Hence, they are useful for passing external data to the QNode. See `Keyword arguments` in [Advanced features](https://pennylane.readthedocs.io/en/latest/tutorials/advanced_usage.html#keyword-arguments) tutorial.

</div>

</div>